<a href="https://colab.research.google.com/github/dr-kinder/playground/blob/dev/colab_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
# Need to upgrade matplotlib for some PyVista plotting commands to work.
# Run this cell, then select "Runtime > Restart runtime" from the CoLab menu.
!pip install --upgrade matplotlib

display.clear_output()

In [ ]:
from IPython import display

In [ ]:
try:
    import gmsh
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh

display.clear_output()

In [ ]:
# This simulation uses real values.
try:
    import dolfinx
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

display.clear_output()

In [ ]:
try:
    import multiphenicsx
except ImportError:
    !pip3 install "multiphenicsx@git+https://github.com/multiphenics/multiphenicsx.git@906a91b"
    import multiphenicsx

display.clear_output()

# Making Movies in CoLab

The cells below illustrate how to make a movie in CoLab using PyVista.  The first cell comes directly from the PyVista documentation.  The original code is at

https://docs.pyvista.org/examples/02-plot/gif.html

The command `pv.start_xvfb()` seems necessary to create graphics with CoLab.  It may not be necessary on other platforms.

Comments have been added to explain the process.

This cell creates a movie by exporting a series of separate plots as images in a GIF.  PyVista does all the processing.  In CoLab, the file will be saved in the current directory: `./`.  It can also be accessed in the `/content/` directory.

The next cell uses IPython's HTML and image display tools to display the GIF in the notebook.

In [ ]:
# https://docs.pyvista.org/examples/02-plot/gif.html
import numpy as np
import matplotlib

import pyvista as pv
# Start virtual framebuffer for plotting.
# Seems essential in CoLab.
pv.start_xvfb()

# Define coordinate and function arrays.
x = np.arange(-10, 10, 0.5)
y = np.arange(-10, 10, 0.5)
x, y = np.meshgrid(x, y)
r = np.sqrt(x**2 + y**2)
z = np.sin(r)

# Create a structured surface.
grid = pv.StructuredGrid(x, y, z)

# Create a plotter object and set the scalars to the Z height.
# The scalar values determine the color of each region.
plotter = pv.Plotter(notebook=False, off_screen=True)
plotter.add_mesh(
    grid,
    scalars=z.ravel(),
    lighting=False,
    show_edges=True,
    scalar_bar_args={"title": "Height"},
    clim=[-1, 1],
)

# Open a gif.
plotter.open_gif("wave.gif")

# Make a copy of the point array.
# Update the z-values for each frame.
pts = grid.points.copy()

# Update Z and write a frame for each update.
nframe = 15
for phase in np.linspace(0, 2 * np.pi, nframe + 1)[:nframe]:
    # Update the points.
    z = np.sin(r + phase)
    pts[:, -1] = z.ravel()

    # Update the plot.
    plotter.update_coordinates(pts, render=False)
    plotter.update_scalars(z.ravel(), render=False)

    # Write a frame. This triggers a render.
    plotter.write_frame()

# Close the plotter and finalize the movie.
plotter.close()

In [ ]:
from IPython.display import Image

# Open the GIF and read it byte by byte.
gif = open('/content/wave.gif', 'rb').read()

# Display it as an image.
Image(gif) 

# Diffusion

The rest of the notebook illustrate how to solve a diffusion problem and visualize the results using FEniCSx.  It was adapted for CoLab from the tutorial for the FEniCS 2022 conference:

https://jorgensd.github.io/fenics22-tutorial/heat_eq.html

In [ ]:
# Import specific libraries from dolfinx.
from dolfinx import mesh, fem, io, plot, la

In [ ]:
# Create a simple rectangular mesh.
from mpi4py import MPI
length, height = 10, 3
Nx, Ny = 80, 60
extent = [[0., 0.], [length, height]]
domain = mesh.create_rectangle(
    MPI.COMM_WORLD, extent, [Nx, Ny], mesh.CellType.quadrilateral)

In [ ]:
# If there are multiple processes, distribute the mesh to each.
local_domain = mesh.create_rectangle(
    MPI.COMM_SELF, extent, [Nx, Ny], mesh.CellType.quadrilateral)

In [ ]:
# Prepare the mesh for plotting.
import dolfinx.plot
topology, cells, geometry = dolfinx.plot.create_vtk_mesh(domain)

In [ ]:
# Plot the mesh with PyVista.
import pyvista
pyvista.start_xvfb()

# Turn the mesh into a PyVista grid.
grid = pyvista.UnstructuredGrid(topology, cells, geometry)
pyvista.set_jupyter_backend("pythreejs")

# Create the plot and export it to HTML.
plotter = pyvista.Plotter(window_size=(800, 400))
renderer = plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
plotter.camera.zoom(2)

# Save the HTML file.
plotter.export_html("./beam.html", backend="pythreejs")

In [ ]:
# Use the IPython library to embed the HTML in the CoLab notebook.
import IPython
IPython.display.HTML(filename='/content/beam.html')

In [ ]:
# Now set up the finite element problem.
# ufl gives us the tools for an abstract description of the problem.
from ufl import (TestFunction, SpatialCoordinate, TrialFunction,
                 as_vector, dx, grad, inner, system)

# These are the functions and function spaces for the finite element problem.
V = fem.FunctionSpace(domain, ("Lagrange", 1))
u = TrialFunction(V)
v = TestFunction(V)
un = fem.Function(V)

# This is the source.  We set it to zero for a diffusion equation whose
# sources are only on the boundary.
f = fem.Constant(domain, 0.0)

# This is the diffusion coefficient.
mu = fem.Constant(domain, 100.0)

# This is the time step for the simulation.
dt = fem.Constant(domain, 0.05)

In [ ]:
# This is the problem we are going to solve.
F = inner(u - un, v) * dx + dt * mu * inner(grad(u), grad(v)) * dx
F -= dt * inner(f, v) * dx
(a, L) = system(F)

In [ ]:
# Define the potential on one boundary.
# The other boundaries will be open.
import numpy as np

def uD_function(t):
    return lambda x: np.exp(-(x[1] - height/2)**2) * np.cos(t)
    # return lambda x: x[1] * np.cos(t)

# Turn this function definition into a time-dependent function on the mesh.
uD = fem.Function(V)

# Initialize the function for t=0.
t = 0
uD.interpolate(uD_function(t))

In [ ]:
# Define a function to locate boundaries.
# It will return True for points on the boundary, and False otherwise.
def dirichlet_facets(x):
    return np.isclose(x[0], length)

# Next, we use this function to tag each cell along the boundary.
tdim = domain.topology.dim
bc_facets = mesh.locate_entities_boundary(
    domain, tdim - 1, dirichlet_facets)

# Identify those for which the function was True as
# boundary_dofs = "boundary degrees of freedom"
bndry_dofs = fem.locate_dofs_topological(V, tdim - 1, bc_facets)

# Add these to the list of boundary conditions to be imposed.
# This will apply the uD function to the cells along the boundary.
# It will be updated during each time step.
bcs = [fem.dirichletbc(uD, bndry_dofs)]

In [ ]:
# Building the matrix for the finite element problem takes a while.
# It does not change throughout the problem, either.  The following
# line build the matrix once and store it for reuse.
compiled_a = fem.form(a)
A = fem.petsc.assemble_matrix(compiled_a, bcs=bcs)
A.assemble()

In [ ]:
# We do the same for the vector, but it will be updated at each time step.
compiled_L = fem.form(L)
b = fem.Function(V)

In [ ]:
# Now we use the PETSc problem to construct the linear algebra problem.
from petsc4py import PETSc
solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.CG)
pc = solver.getPC()
pc.setType(PETSc.PC.Type.HYPRE)
pc.setHYPREType("boomeramg")

In [ ]:
# We will create a GIF using PyVista again.
# Start virtual framebuffer for plotting.
import pyvista
pyvista.start_xvfb()

# Create a plotter, but don't display the plot.
# It will be making a movie in the background.
plotter = pyvista.Plotter(notebook=False, off_screen=True)
plotter.open_gif("u_time.gif")

In [ ]:
# This is the function we will update.
uh = fem.Function(V)

# Prepare the mesh for PyVista ...
topology, cells, geometry = plot.create_vtk_mesh(V)

# And add it to the plot.
grid = pyvista.UnstructuredGrid(topology, cells, geometry)
grid.point_data["uh"] = uh.x.array

In [ ]:
# The grid is flat.  This will warp the grid into three dimensions
# using the value of the function as the height.
warped = grid.warp_by_scalar("uh", factor=2)

# Add the warped grid to the plot.
plotter.add_mesh(
    warped,
    lighting=False,
    show_edges=False,
    scalar_bar_args={"title": "Height"},
    clim=[-1, 1],
    cmap='turbo'
)

# Set up the camera for the movie.
# plotter.view_xy()
# plotter.camera.zoom(-1.3)

In [ ]:
# Now, compute an update for each time step and add it to the movie.

# Initial time
t = 0

# Final time
T = 4 * np.pi

# Loop until completion.
while t < T:
    # Update boundary conditions.
    t += dt.value
    uD.interpolate(uD_function(t))

    # Assemble the RSH: the vector on the "right-hand side" of A.u = b.
    b.x.array[:] = 0
    fem.petsc.assemble_vector(b.vector, compiled_L)

    # Apply boundary condition.
    # These commands distribute the updated problem to all processes.
    fem.petsc.apply_lifting(b.vector, [compiled_a], [bcs])
    b.x.scatter_reverse(la.ScatterMode.add)
    fem.petsc.set_bc(b.vector, bcs)

    # Solve linear problem.
    solver.solve(b.vector, uh.vector)

    # Distribute the solution to all processes.
    uh.x.scatter_forward()

    # Update un --- the current value of the function.
    un.x.array[:] = uh.x.array

    # Update the plot and save the frame to the GIF.
    grid.point_data["uh"] = uh.x.array
    warped = grid.warp_by_scalar("uh", factor=2)
    plotter.update_scalars(uh.x.array, render=False)
    plotter.update_coordinates(warped.points)
    plotter.write_frame()

# Close the plotter and finish processing the movie.
plotter.close()

In [ ]:
# Read the GIF.  Display it in the notebook.
from IPython.display import Image
Image(open('/content/u_time.gif', 'rb').read())